# Necessary Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import sys
import argparse
import csv
from scipy.sparse import *
import pandas as pd
import regex 
from sklearn.base import TransformerMixin
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.util import ngrams
from sklearn.model_selection import StratifiedKFold, train_test_split


In [4]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
import re

In [1]:
!pip install emoji==0.6.0

     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp37-none-any.whl size=49716 sha256=0e166aa950fc366fd2399964457c95f2b06823cf2cd785edca1a2228a1df05f0
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [5]:
!pip  install Arabic-Stopwords


     |████████████████████████████████| 358kB 7.5MB/s 
     |████████████████████████████████| 112kB 14.6MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp37-none-any.whl size=113324 sha256=a6a5e94e9f664015f89a3891c65d7762037cdc52d296e30c1518288b26899fce
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic


In [6]:
import arabicstopwords.arabicstopwords as stp

In [7]:
import emoji


In [8]:
from collections import Counter, OrderedDict


In [9]:
import math

In [10]:
import random
from random import seed

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE


In [28]:
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora


In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
import nltk
from nltk.corpus import stopwords
from lightgbm import LGBMClassifier
import numpy as np
from gensim.models import KeyedVectors


Mount Drive

In [12]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")

Mounted at /content/drive


# Helper Functions for preprocessing

In [16]:
def load_w2v(filepath,binary):
    return KeyedVectors.load_word2vec_format(filepath, binary=binary)

In [19]:
stop_words = pd.Series(stopwords.words('arabic'))


In [21]:
def clean_text(text):  
    # ref: https://github.com/bakrianoo/aravec
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(longation, subst, text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()


    
    return text

In [22]:
# Get the vec representation of a set of tweets based on a specified embedding (can be a word or emoji mapping)
def get_tweets_embeddings(tweets, vec_map, embedding_dim=100, init_unk=False, variance=None, weighted_average=True):
    # Get the variance of the embedding map
    if init_unk and variance is None:
        variance = embedding_variance(vec_map)
        print("Vector mappings have variance ", variance)
    # If set, calculate the tf-idf weight of each embedding, otherwise, no weighting (all weights are 1.0)
    if weighted_average:
        weights = get_tf_idf_weights(tweets, vec_map)
    else:
        weights = {k: 1.0 for k in vec_map.vocab}
    tw_emb = np.zeros((len(tweets), embedding_dim))
    for i, tw in enumerate(tweets):
        total_valid = 0
        for word in tw.split():
            embedding_vector=None 
            if word in vec_map.wv:
              embedding_vector = vec_map[word]
            if embedding_vector is not None:
                tw_emb[i] = tw_emb[i] + embedding_vector * weights[word]
                total_valid += 1
            elif init_unk:
                seed(1337603)
                tw_emb[i] = np.random.uniform(-variance, variance, size=(1, embedding_dim))
            # else:
            #    print("Not found: ", word)
        # Get the average embedding representation for this tweet
        tw_emb[i] /= float(max(total_valid, 1))
    return tw_emb


In [23]:
# Calculate the variance of an embedding (like glove, word2vec, emoji2vec, etc)
# Used to sample new uniform distributions of vectors in the interval [-variance, variance]
def embedding_variance(vec_map):
    variance = np.sum([np.var(vec) for vec in vec_map.vectors]) / len(vec_map.vectors)
    return variance


In [24]:
# Get the tf-idf weighting scheme (used to measure the contribution of a word in a tweet => weighted sum of embeddings)
def get_tf_idf_weights(tweets, vec_map):
    df = {}
    for tw in tweets:
        words = set(tw.split())
        for word in words:
            if word not in df:
                df[word] = 0.0
            df[word] += 1.0
    idf = OrderedDict()
    for word in vec_map.vocab:
        n = 1.0
        if word in df:
            n += df[word]
        score = math.log(len(tweets) / float(n))
        idf[word] = score
    return idf


# Compute the similarity of 2 vectors, both of shape (n, )
def cosine_similarity(u, v):
    dot = np.dot(u, v)
    norm_u = np.sqrt(np.sum(u ** 2))
    norm_v = np.sqrt(np.sum(v ** 2))
    cosine_distance = dot / (norm_u * norm_v)
    return cosine_distance


# Convert emojis to unicode representations by removing any variation selectors
# Info: http://www.unicode.org/charts/PDF/UFE00.pdf
def convert_emoji_to_unicode(emoji):
    unicode_emoji = emoji.encode('unicode-escape')
    find1 = unicode_emoji.find(b"\\ufe0f")
    unicode_emoji = unicode_emoji[:find1] if find1 != -1 else unicode_emoji
    find2 = unicode_emoji.find(b"\\ufe0e")
    unicode_emoji = unicode_emoji[:find2] if find2 != -1 else unicode_emoji
    return unicode_emoji


# Performs the word analogy task: a is to b as c is to ____.
def make_analogy(a, b, c, vec_map):
    a = convert_emoji_to_unicode(a)
    b = convert_emoji_to_unicode(b)
    c = convert_emoji_to_unicode(c)

    e_a, e_b, e_c = vec_map[a], vec_map[b], vec_map[c]

    max_cosine_sim = -100
    best = None
    best_list = {}
    for v in vec_map.keys():
        # The best match shouldn't be one of the inputs, so pass on them.
        if v in [a, b, c]:
            continue
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)
        cosine_sim = cosine_similarity(e_b - e_a, vec_map[v] - e_c)
        best_list[v] = cosine_sim
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best = v
    sorted_keys = sorted(best_list, key=best_list.get, reverse=True)
    print("Top 5 most similar emojis: ", [r.decode('unicode-escape') for r in sorted_keys[:5]])
    print(str.format('{} - {} + {} = {}', a.decode('unicode-escape'), b.decode('unicode-escape'),
                     c.decode('unicode-escape'), best.decode('unicode-escape')), "\n\n")


# Get the Euclidean distance between two vectors
def euclidean_distance(u_vector, v_vector):
    distance = np.sqrt(np.sum([(u - v) ** 2 for u, v in zip(u_vector, v_vector)]))
    return distance


# Given a tweet, return the scores of the most similar/dissimilar pairs of words
def get_similarity_measures(tweet, vec_map, weighted=False, verbose=True):
    # Filter a bit the tweet so that no punctuation and no stopwords are included
    stopwords = data_proc.get_stopwords_list()
    filtered_tweet = list(set([w.lower() for w in tweet.split()
                               if w.isalnum() and w not in stopwords and w.lower() in vec_map.keys()]))
    # Compute similarity scores between any 2 words in filtered tweet
    similarity_scores = []
    max_words = []
    min_words = []
    max_score = -100
    min_score = 100
    for i in range(len(filtered_tweet) - 1):
        wi = filtered_tweet[i]
        for j in range(i + 1, len(filtered_tweet)):
            wj = filtered_tweet[j]
            similarity = cosine_similarity(vec_map[wi], vec_map[wj])
            if weighted:
                similarity /= euclidean_distance(vec_map[wi], vec_map[wj])
            similarity_scores.append(similarity)
            if max_score < similarity:
                max_score = similarity
                max_words = [wi, wj]
            if min_score > similarity:
                min_score = similarity
                min_words = [wi, wj]
    if verbose:
        print("Filtered tweet: ", filtered_tweet)
        if max_score != -100:
            print("Maximum similarity is ", max_score, " between words ", max_words)
        else:
            print("No max! Scores are: ", similarity_scores)
        if min_score != 100:
            print("Minimum similarity is ", min_score, " between words ", min_words)
        else:
            print("No min! Scores are: ", similarity_scores)
    return max_score, min_score


# Custom metric function adjusted from https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def f1_score(y_true, y_pred):
    # Recall metric. Only computes a batch-wise average of recall,
    # a metric for multi-label classification of how many relevant items are selected.
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    # Precision metric. Only computes a batch-wise average of precision,
    # a metric for multi-label classification of how many selected items are relevant.
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision*recall) / (precision+recall))


# This code allows you to see the mislabelled examples
def analyse_mislabelled_examples(x_test, y_test, y_pred):
    for i in range(len(y_test)):
        num = np.argmax(y_pred[i])
        if num != y_test[i]:
            print('Expected:', y_test[i], ' but predicted ', num)
            print(x_test[i])
# Extract each tweet's emojis - obv. it's just a brute force solution (so, it's slow) but works in ALL cases
def extract_emojis(tweets):
    emojis = []
    for tw in tweets:
        tw_emojis = []
        for word in tw:
            chars = list(word)
            for ch in chars:
                if ch in emoji.UNICODE_EMOJI:
                    tw_emojis.append(ch)
        emojis.append(' '.join(tw_emojis))
    return emojis

In [25]:
# Get some idea about the max and mean length of the tweets (useful for deciding on the sequence length)
def get_max_len_info(tweets, average=False):
    sum_of_length = sum([len(l.split()) for l in tweets])
    avg_tweet_len = sum_of_length / float(len(tweets))
    print("Mean of train tweets: ", avg_tweet_len)
    max_tweet_len = len(max(tweets, key=len).split())
    print("Max tweet length is = ", max_tweet_len)
    if average:
        return avg_tweet_len
    return max_tweet_len

In [26]:
def encode_text_as_word_indexes(train_tweets,  max_num_words=None, lower=False, char_level=False):
    # Create the tokenizer
    tokenizer = Tokenizer(num_words=max_num_words, filters='', lower=lower, split=" ", char_level=char_level)
    # Fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_tweets)
    # Encode each example as a sequence of word indexes based on the vocabulary of the tokenizer
    x_train = tokenizer.texts_to_sequences(train_tweets)
   
    return tokenizer, x_train

In [27]:
def create_bag_of_words(df):
  wordfreq = {}
  sentence_vectors = []
  for comment in df:
    corpus = nltk.sent_tokenize(comment)
    for i in range(len(corpus )):
      corpus [i] = corpus [i].lower()
      corpus [i] = re.sub(r'\W',' ',corpus [i])
      corpus [i] = re.sub(r'\s+',' ',corpus [i])
    
    for sentence in corpus:
        tokens = nltk.word_tokenize(sentence)
        for token in tokens:
            if token not in wordfreq.keys():
                wordfreq[token] = 1
            else:
                wordfreq[token] += 1
  
    for sentence in corpus:
        sentence_tokens = nltk.word_tokenize(sentence)
        sent_vec = []
        for token in wordfreq:
            if token in sentence_tokens:
                sent_vec.append(1)
            else:
                sent_vec.append(0)
        sentence_vectors.append(sent_vec)
  return np.asarray(sentence_vectors)


In [29]:
def sent_to_words(sentences):
    # for sentence in sentences:
        # deacc=True removes punctuations
    yield(gensim.utils.simple_preprocess(str(sentences), deacc=True))

In [30]:
def topic_modelling(df,stop_words):
  def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
  def remove_stopwords(texts):
      words=[]
      for w in texts:
        # print(w)
        small_list=[]
        for www in w:
          if www not in stop_words.tolist():
            small_list.append(www)
        words.append(small_list)
      return  words
 
  data = df.cleaned_text.values.tolist()
  data_words = list(sent_to_words(data))

  # remove stop words
  data_words = remove_stopwords(data_words)
  # Create Dictionary
  id2word = corpora.Dictionary(data_words)
  # Create Corpus
  texts = data_words
  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]
  # number of topics
  num_topics = 1000
  # Build LDA model
  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                        id2word=id2word,
                                        num_topics=num_topics)
  return lda_model,id2word
  
  


In [34]:
def get_key(val,my_dict):
    for key, value in my_dict.items():
         if val == value:
             return key

# Load Data and preprocess

Load Data

In [14]:
train_data=pd.read_csv('')#### load train data csv file

In [15]:
test_data=pd.read_csv('')#### load test data csv file


preprocess data

In [31]:
#### apply preprocessing as removal of URLs, stopwords, emojis, numbers, english alphabet, text normalization
train_data['cleaned_text'] = train_data.tweet.apply(clean_text)
test_data['cleaned_text'] = test_data.tweet.apply(clean_text)


In [39]:
# Extract just the emojis present in each tweet in each set
x_train_emoji = extract_emojis(train_data['tweet'])




In [40]:
x_test_emoji=extract_emojis(test_data['tweet'])

Extract TF-IDF feature and Count vector

In [32]:
vectorizer = TfidfVectorizer( use_idf =True,stop_words = stop_words.tolist(), analyzer='word', lowercase=False, sublinear_tf=True)
X_Train_tfidf = vectorizer.fit_transform(train_data['cleaned_text'].ravel())
X_Test_tfidf = vectorizer.transform(test_data['cleaned_text'].ravel())

In [33]:
count_vect = CountVectorizer(stop_words = stop_words.tolist())
X_Train_count = count_vect.fit_transform(train_data['cleaned_text'])
X_Test_count = count_vect.transform(test_data['cleaned_text'])

Load Embeddings

In [37]:
word2vec_map = load_w2v("/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/arabic-news.bin", binary=True)

In [41]:
emoji2vec_map = load_w2v("/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/emoji2vec.bin",True)


In [42]:
init_unk = True
var = None
weighted = True

Apply vector Mappings

In [43]:
x_train_emoji_emb = get_tweets_embeddings(x_train_emoji, emoji2vec_map, EMBEDDING_DIM,
                                                init_unk=init_unk, variance=var, weighted_average=weighted)
x_train_word_emb = get_tweets_embeddings(train_data['cleaned_text'], word2vec_map, EMBEDDING_DIM,
                                               init_unk=init_unk, variance=var, weighted_average=weighted)

Vector mappings have variance  0.0033188830794231637


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Vector mappings have variance  0.7012621721375845


In [44]:
x_test_emoji_emb = get_tweets_embeddings(x_test_emoji, emoji2vec_map, EMBEDDING_DIM,
                                                init_unk=init_unk, variance=var, weighted_average=weighted)
x_test_word_emb = get_tweets_embeddings(test_data['cleaned_text'], word2vec_map, EMBEDDING_DIM,
                                               init_unk=init_unk, variance=var, weighted_average=weighted)

Vector mappings have variance  0.0033188830794231637


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Vector mappings have variance  0.7012621721375845


Concatenate featuers

In [45]:

# Obtain features by concatenating word embeddings with all emoji embeddings
x_train_features_concat = []
for t, e in zip(x_train_word_emb, x_train_emoji_emb):
    x_train_features_concat.append(np.concatenate((t, e), axis=0))



# Obtain features by adding together the word embeddings with all emoji embeddings
x_train_features_sum = []
for t, e in zip(x_train_word_emb, x_train_emoji_emb):
    x_train_features_sum.append(t + e)



In [48]:

# Obtain features by concatenating word embeddings with all emoji embeddings
x_test_features_concat = []
for t, e in zip(x_test_word_emb, x_test_emoji_emb):
    x_test_features_concat.append(np.concatenate((t, e), axis=0))



# Obtain features by adding together the word embeddings with all emoji embeddings
x_test_features_sum = []
for t, e in zip(x_test_word_emb, x_test_emoji_emb):
    x_test_features_sum.append(t + e)



In [49]:
# Transform the output into categorical data or just keep it as it is (in a numpy array)
train_labels_task1 = to_categorical(np.asarray(train_data['sarcasm']))
train_labels_task2 = to_categorical(np.asarray(train_data['sentiment'].factorize()[0]))




In [50]:
# Transform the output into categorical data or just keep it as it is (in a numpy array)
test_labels_task1 = to_categorical(np.asarray(test_data['sarcasm']))
test_labels_task2 = to_categorical(np.asarray(test_data['sentiment'].factorize()[0]))




## ML Based approach

In [51]:

def print_statistics(y, y_pred):
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'C': C_range}
    linear_svm = LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'kernel': ['rbf'], 'C': C_range, 'gamma': gamma_range}
    nonlinear_svm = SVC(class_weight=class_ratio)
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'C': C_range}
    log_regr = LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2')
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    print_statistics(y_test, y_hat)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2')
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = LinearSVC(C=0.1, penalty='l2')
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)

In [52]:
def get_classes_ratio_as_dict(labels):
    ratio = Counter(labels)
    ratio_dict = {0: float(max(ratio[0], ratio[1]) / ratio[0]), 1: float(max(ratio[0], ratio[1]) / ratio[1])}
    print('Class ratio: ', ratio_dict)
    return ratio_dict

### Accuracy Sentiment detection

In [58]:
linear_svm(x_train_features_concat,train_data['sentiment'],x_test_features_concat,test_data['sentiment'])


          Linear SVM

Accuracy: 0.552
Precision: 0.530
Recall: 0.552
F_score: 0.537

              precision    recall  f1-score   support

         NEG       0.63      0.73      0.68      1677
         NEU       0.36      0.27      0.31       748
         POS       0.44      0.39      0.42       575

    accuracy                           0.55      3000
   macro avg       0.48      0.46      0.47      3000
weighted avg       0.53      0.55      0.54      3000



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [60]:
linear_svm(x_train_features_sum,train_data['sentiment'],x_test_features_sum,test_data['sentiment'])


          Linear SVM

Accuracy: 0.539
Precision: 0.517
Recall: 0.539
F_score: 0.524

              precision    recall  f1-score   support

         NEG       0.62      0.72      0.67      1677
         NEU       0.36      0.27      0.31       748
         POS       0.41      0.35      0.38       575

    accuracy                           0.54      3000
   macro avg       0.46      0.45      0.45      3000
weighted avg       0.52      0.54      0.52      3000



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [61]:
logistic_regression(x_train_features_sum,train_data['sentiment'],x_test_features_sum,test_data['sentiment'])


 Logistic Regression

Accuracy: 0.517
Precision: 0.509
Recall: 0.517
F_score: 0.504

              precision    recall  f1-score   support

         NEG       0.63      0.67      0.65      1677
         NEU       0.37      0.20      0.26       748
         POS       0.35      0.49      0.40       575

    accuracy                           0.52      3000
   macro avg       0.45      0.45      0.44      3000
weighted avg       0.51      0.52      0.50      3000



In [63]:
logistic_regression(x_train_features_concat,train_data['sentiment'],x_test_features_concat,test_data['sentiment'])


 Logistic Regression

Accuracy: 0.531
Precision: 0.515
Recall: 0.531
F_score: 0.515

              precision    recall  f1-score   support

         NEG       0.63      0.70      0.66      1677
         NEU       0.36      0.20      0.26       748
         POS       0.37      0.48      0.42       575

    accuracy                           0.53      3000
   macro avg       0.46      0.46      0.45      3000
weighted avg       0.52      0.53      0.51      3000



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Accuracy Sarcsam detection 

In [55]:
linear_svm(x_train_features_concat,train_data['sarcasm'],x_test_features_concat,test_data['sarcasm'],get_classes_ratio_as_dict(train_data['sarcasm']))

Class ratio:  {0: 1.0, 1: 4.787822878228782}

          Linear SVM

Accuracy: 0.517
Precision: 0.701
Recall: 0.517
F_score: 0.532

              precision    recall  f1-score   support

       False       0.84      0.41      0.55      2179
        True       0.34      0.79      0.47       821

    accuracy                           0.52      3000
   macro avg       0.59      0.60      0.51      3000
weighted avg       0.70      0.52      0.53      3000



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [53]:
linear_svm(x_train_features_sum,train_data['sarcasm'],x_test_features_sum,test_data['sarcasm'],get_classes_ratio_as_dict(train_data['sarcasm']))

Class ratio:  {0: 1.0, 1: 4.787822878228782}

          Linear SVM

Accuracy: 0.486
Precision: 0.695
Recall: 0.486
F_score: 0.496

              precision    recall  f1-score   support

       False       0.83      0.37      0.51      2179
        True       0.32      0.81      0.46       821

    accuracy                           0.49      3000
   macro avg       0.58      0.59      0.49      3000
weighted avg       0.69      0.49      0.50      3000



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [54]:
logistic_regression(x_train_features_sum,train_data['sarcasm'],x_test_features_sum,test_data['sarcasm'],get_classes_ratio_as_dict(train_data['sarcasm']))

Class ratio:  {0: 1.0, 1: 4.787822878228782}

 Logistic Regression

Accuracy: 0.492
Precision: 0.687
Recall: 0.492
F_score: 0.504

              precision    recall  f1-score   support

       False       0.82      0.38      0.52      2179
        True       0.32      0.78      0.46       821

    accuracy                           0.49      3000
   macro avg       0.57      0.58      0.49      3000
weighted avg       0.69      0.49      0.50      3000



In [56]:
logistic_regression(x_train_features_concat,train_data['sarcasm'],x_test_features_concat,test_data['sarcasm'],get_classes_ratio_as_dict(train_data['sarcasm']))

Class ratio:  {0: 1.0, 1: 4.787822878228782}

 Logistic Regression

Accuracy: 0.519
Precision: 0.700
Recall: 0.519
F_score: 0.535

              precision    recall  f1-score   support

       False       0.84      0.42      0.56      2179
        True       0.34      0.78      0.47       821

    accuracy                           0.52      3000
   macro avg       0.59      0.60      0.51      3000
weighted avg       0.70      0.52      0.53      3000



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
